In [ ]:
import torch

from datetime import datetime
from pprint import pprint

from config import *
from data.extract import DetectorDataProvider, LookUpTable
from data.util import count_points_in_period

%reload_ext autoreload
%autoreload 2

### Read data from the data set

In [ ]:
lut = LookUpTable('Data')

Check list of all sections

In [ ]:
sections = lut.get_sections()
pprint(list(sections)[:10])
print(len(sections))

Extract list of detectors for each section

In [ ]:
int_det = lut.get_detectors_per_section()
int_det

Extract counts for all sections

In [ ]:
ddp = DetectorDataProvider('Data')

def get_counts_entering_section(sec):
    try:
        return ddp.get_counts_entering_section(sec['End'], sec['Detectors'], CONFIG.read_period)
    except KeyError as e:
        print(sec['End'], sec['Detectors'])
        raise e

int_det['Counts'] = int_det.apply(get_counts_entering_section, axis=1)
int_det

### Write data into a matrix

Write counts into a matrix
$$Q=[q_1,q_2,...,q_p]$$
for
$$q_i=\set{z(s_i,t_1),z(s_i,t_2),...z(s_i,t_d)}^T,$$
$z(s_i,t_j)$ is the traffic flow of the road section $s_i$ within the time interval $(t_0,t_j)$

In [ ]:
mat_q = torch.tensor(int_det['Counts'].tolist()).T
CONFIG.save(mat_q, 'mat_q.pt')
assert mat_q.shape == (count_points_in_period(CONFIG.read_period), 2 * len(sections))
mat_q.shape                                                         